In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)  # randomness of response

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Korea")

chat.predict(prompt)

"The distance between Mexico and Korea varies depending on the specific locations being referred to. \n\nIf we consider the distance between Mexico City, Mexico, and Seoul, South Korea, the approximate straight-line distance is around 11,000 kilometers or 6,835 miles.\n\nHowever, if we consider the distance between the closest points on the two countries' coastlines, the distance is significantly shorter. The distance between the westernmost point of Mexico (Cabo San Lucas) and the easternmost point of South Korea (Busan) is approximately 10,000 kilometers or 6,214 miles.\n\nIt's important to note that these distances are approximate and can vary depending on the specific locations being compared."

In [2]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek", name="Socrates", country_a="Mexico", country_b="Korea"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Κορέας εξαρτάται από το ποια πόλη του Μεξικού και ποια πόλη της Κορέας αναφέρεσαι. Συνήθως, η απόσταση ανάμεσά τους είναι περίπου 13.000 χιλιόμετρα.')